<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Asset Management

### Asset Allocation Based on Predictions

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

## Expected Return Predictions

Topics of interest include:

* deep learning-based predictions
* equally weighted portolio as benchmark
* prediction-weighted portfolio

# Imports & Configurations

In [ ]:
!git clone https://github.com/tpq-classes/python_for_asset_management.git
import sys
sys.path.append('python_for_asset_management')


In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
from pylab import mpl, plt

In [ ]:
np.random.seed(1000)
plt.style.use('seaborn-v0_8')
pd.set_option('display.precision', 4)
pd.set_option('display.float_format', '{:.4f}'.format)
np.set_printoptions(suppress=True, precision=4)
%config InlineBackend.figure_format = 'svg'

## The Data

In [ ]:
raw = pd.read_csv('http://hilpisch.com/aiif_eikon_eod_data.csv',
                      index_col=0, parse_dates=True)

In [ ]:
noa = 3

In [ ]:
symbols = raw.columns[:noa]
symbols

In [ ]:
# symbols = ['SPY', 'GLD', 'EUR=']

In [ ]:
data = pd.DataFrame(raw[symbols])

In [ ]:
rets = np.log(data / data.shift(1))

In [ ]:
rets.dropna(inplace=True)

In [ ]:
rets.tail()

In [ ]:
lags = 3

In [ ]:
def create_lags(rets):
    global cols
    cols = []
    for sym in rets.columns:
        for lag in range(1, lags + 1):
            col = f'{sym}_lag_{lag}'
            rets[col] = rets[sym].shift(lag)
            cols.append(col)

In [ ]:
create_lags(rets)

In [ ]:
cols

In [ ]:
rets.dropna(inplace=True)

## Predicting Returns

In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
model = MLPRegressor(hidden_layer_sizes=2 * [32,],
                    random_state=100, shuffle=False)

In [ ]:
weeks = pd.date_range(rets.index[0], rets.index[-1], freq='w')

In [ ]:
weeks

In [ ]:
start = 51

In [ ]:
%%time
rets_ = pd.DataFrame()
for w in weeks[start:-1]:
    train = rets.loc[:w].copy()
    mu, std = train.mean(), train.std()
    train_ = (train - mu) / std
    model.fit(train_[cols], train[symbols])
    test = rets.loc[w:w + weeks.freq].copy()
    test_ = (test - mu) / std
    rets_ = pd.concat([rets_, pd.DataFrame(model.predict(test_[cols]),
                                  columns=symbols, index=test.index)])  

In [ ]:
train.index[0], train.index[-1]

In [ ]:
test.index[0], test.index[-1]

In [ ]:
rets_.head()

## Equally Weighted Portfolio

In [ ]:
strat = np.sign(rets_) * rets.loc[rets_.index][symbols]

In [ ]:
# single assets
strat.cumsum().apply(np.exp).plot();  # out-of-sample performance

In [ ]:
phi = np.ones(noa) / noa  # equal weight portfolio

In [ ]:
# equally weighted
(strat * phi).cumsum().sum(axis=1).apply(np.exp).plot(label='Portfolio')
plt.legend();  # out-of-sample performance

## Prediction-Weighted Portfolios

In [ ]:
w = (rets_.iloc[:].T / rets_.sum(axis=1)).T

In [ ]:
w.head()

In [ ]:
w.head().sum(axis=1)

In [ ]:
w.iloc[50:75].plot();

In [ ]:
rets[symbols].loc[w.index].head()

In [ ]:
w.head()

In [ ]:
strat = w * rets[symbols].loc[w.index]

In [ ]:
strat.head()

In [ ]:
# single assets
strat.cumsum().apply(np.exp).plot(label='Portfolio')
plt.legend();  # out-of-sample performance

In [ ]:
# prediction-weighted portfolio
strat.cumsum().loc[:'2020-7-1'].sum(axis=1).apply(np.exp).plot(label='Portfolio')
plt.legend();  # out-of-sample performance

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>